In [ ]:
# A) Setup & artifacts  
# =========================

import os
import json
import joblib
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEFAULT_DTYPE = torch.float32
torch.set_default_dtype(DEFAULT_DTYPE)

#model architecture
class MLP(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_neurons, dropout_rate=0.2):
        super().__init__()
        self.fc1 = nn.Linear(input_dim, hidden_neurons)
        self.fc2 = nn.Linear(hidden_neurons, hidden_neurons)
        self.fc3 = nn.Linear(hidden_neurons, hidden_neurons)
        self.fc7 = nn.Linear(hidden_neurons, output_dim)
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, x):
        hidden = F.leaky_relu(self.fc1(x)); hidden = self.dropout(hidden)
        hidden = F.leaky_relu(self.fc2(hidden)); hidden = self.dropout(hidden)
        hidden = F.leaky_relu(self.fc3(hidden)); hidden = self.dropout(hidden)
        out = self.fc7(hidden)
        return out

ARTIFACT_DIR   = "artifacts"
MODEL_PATH     = os.path.join(ARTIFACT_DIR, "mlp_boiler_state_dict.pth")
CONFIG_PATH    = os.path.join(ARTIFACT_DIR, "model_config.json")
X_SCALER_PATH  = os.path.join(ARTIFACT_DIR, "X_scaler_global.joblib")  
Y_SCALER_PATH  = os.path.join(ARTIFACT_DIR, "Y_scaler_global.joblib")  

with open(CONFIG_PATH, "r") as f:
    config = json.load(f)

input_width        = int(config["input_width"])
output_width       = int(config["output_width"])
num_input_features = int(config["num_input_features"])
hidden_units       = int(config["hidden_neurons"])
output_dim         = int(config["output_dim"])  
input_dim          = input_width * num_input_features 

#instantiate model and load weights
model = MLP(input_dim=input_dim, output_dim=output_dim, hidden_neurons=hidden_units, dropout_rate=0.2).to(device)
state_dict = torch.load(MODEL_PATH, map_location=device)
model.load_state_dict(state_dict)
model.eval()

X_scaler = joblib.load(X_SCALER_PATH) 
Y_scaler = joblib.load(Y_SCALER_PATH)  


# B) Data shaping & scaling helpers
# =========================

#column names
input_names = [
    "Coal1","Coal2","Coal3","Coal4","Coal5",
    "PA_Flow","SA_Flow",
    "OFA_p1","OFA_p2","OFA_p3","OFA_p4",
    "T_amb"
]
target_col_Q = "HeatLoad"

def matrix_to_flat(X: np.ndarray) -> np.ndarray:
    X = np.asarray(X)
    return X.reshape(-1)

#MinMaxScaler parameters
_mm_scale  = X_scaler.scale_
_mm_minoff = X_scaler.min_

def phys_to_scaled_feature(i: int, x_phys: np.ndarray) -> np.ndarray:
    return x_phys * _mm_scale[i] + _mm_minoff[i]

def scaled_to_phys_feature(i: int, x_scaled: np.ndarray) -> np.ndarray:
    return (x_scaled - _mm_minoff[i]) / _mm_scale[i]

def robust_scale_Q_phys(Q_phys: np.ndarray) -> np.ndarray:
    center = Y_scaler.center_[0]
    scale  = Y_scaler.scale_[0]
    return (Q_phys - center) / scale


#load dummy inputs & desired heat load (Q_star)
SIM_CSV = "Sim_Test5_MULTIPLE.csv"
df_sim = pd.read_csv(SIM_CSV)

#extract physical arrays
X_phys_full  = df_sim[input_names].to_numpy(dtype=float)           
Q_star_phys  = df_sim[target_col_Q].to_numpy(dtype=float)         
N_STEPS, N_FEATURES_FULL = X_phys_full.shape

# Indices
name_to_idx = {name: i for i, name in enumerate(input_names)}
i_Tamb = name_to_idx["T_amb"]
decision_feature_indices = [i for i in range(N_FEATURES_FULL) if i != i_Tamb]  # remove T_amb
N_DECISION_FEATURES = len(decision_feature_indices)  # 11

#fixed ambient (scaled)
Tamb_fixed_phys   = 295.0
Tamb_fixed_scaled = float(phys_to_scaled_feature(i_Tamb, np.array([Tamb_fixed_phys]))[0])

#scale decision features with MinMax per-feature
X_scaled_decision = np.empty((N_STEPS, N_DECISION_FEATURES), dtype=float)
for k, j in enumerate(decision_feature_indices):
    X_scaled_decision[:, k] = phys_to_scaled_feature(j, X_phys_full[:, j])

#targets in scaled Y-space
yQ_target_scaled   = robust_scale_Q_phys(Q_star_phys)                              

def reconstruct_full_scaled_single(x_dec_scaled_row: np.ndarray) -> np.ndarray:
    X_full = np.empty((N_FEATURES_FULL,), dtype=float)
    for k, j in enumerate(decision_feature_indices):
        X_full[j] = x_dec_scaled_row[k]
    X_full[i_Tamb] = Tamb_fixed_scaled
    return X_full

def hard_project_coal_decision_scaled_row(x_dec_scaled_row: np.ndarray) -> np.ndarray:
    #row-wise strict coal feasibility in physical units, mapped back to scaled.

    xr = np.array(x_dec_scaled_row, dtype=float, copy=True)
    for kd, j_full in zip(coal_indices_decision, coal_indices_full):
        x_phys = scaled_to_phys_feature(j_full, xr[kd])
        x_phys_proj = 0.0 if (x_phys < coal_threshold_phys) else float(np.clip(x_phys, coal_threshold_phys, coal_hi_phys))
        xr[kd] = phys_to_scaled_feature(j_full, np.array([x_phys_proj]))[0]
    return xr

#coal feature indices in the full 12-col space
coal_indices_full   = [name_to_idx[f"Coal{i}"] for i in range(1, 6)]
coal_threshold_phys = 14.5
coal_hi_phys        = 34.0

coal_indices_decision = [decision_feature_indices.index(j) for j in coal_indices_full]


# C & D) Objective
# =========================

w_Q   = 1
w_eff = 0.000014

PRINT_EVERY = 5 

#DE knobs
strategy = "best1bin"
popsize  = 10
maxiter  = 25
mutation = 0.4
recomb   = 0.6

def objective_single_step(x_dec_scaled_row: np.ndarray, yQ_target_scaled_t: float) -> float:
    
    #Minimize: J = w_Q * (yQ_pred - yQ_target)^2  -  w_eff * yEff_pred

    x_dec_feas = hard_project_coal_decision_scaled_row(x_dec_scaled_row)

    x_full_scaled = reconstruct_full_scaled_single(x_dec_feas)

    #model forward
    with torch.no_grad():
        y_scaled = model(torch.from_numpy(x_full_scaled[None, :]).to(device=device, dtype=DEFAULT_DTYPE)).cpu().numpy()[0]


    yQ_pred_scaled   = float(y_scaled[0])
    yEff_pred_scaled = float(y_scaled[1])

    cost_Q = (yQ_pred_scaled - yQ_target_scaled_t) ** 2
    J = w_Q * cost_Q - w_eff * yEff_pred_scaled
    return float(J)



# E) Bounds (scaled)
# =========================

# Build scaled bounds per decision feature from PHYSICAL limits
physical_bounds = {
    "Coal1": (0.0, 34.0),
    "Coal2": (0.0, 34.0),
    "Coal3": (0.0, 34.0),
    "Coal4": (0.0, 34.0),
    "Coal5": (0.0, 34.0),
    "PA_Flow": (40.0, 200.0),
    "SA_Flow": (300.0, 560.0),
    "OFA_p1": (8.0, 35.0),
    "OFA_p2": (8.0, 35.0),
    "OFA_p3": (8.0, 35.0),
    "OFA_p4": (8.0, 35.0),
    "T_amb": (295.0, 295.0),   
}

bounds_decision_single = []
for j in decision_feature_indices:
    lo_phys, hi_phys = physical_bounds[input_names[j]]
    lo_scaled = float(phys_to_scaled_feature(j, np.array([lo_phys]))[0])
    hi_scaled = float(phys_to_scaled_feature(j, np.array([hi_phys]))[0])
    lo_s, hi_s = (min(lo_scaled, hi_scaled), max(lo_scaled, hi_scaled))
    bounds_decision_single.append((lo_s, hi_s))

lb_single = np.array([b[0] for b in bounds_decision_single], dtype=float)
ub_single = np.array([b[1] for b in bounds_decision_single], dtype=float)


# F) Optimiser run — Differential Evolution (segment-wise, minimal)
# =========================

from scipy.optimize import differential_evolution

# --- Segment detection on PHYSICAL HeatLoad targets ---
def find_constant_segments(Q_phys: np.ndarray, rel_tol=2e-3, abs_tol=0.05):
    Q = np.asarray(Q_phys, dtype=float)
    if Q.size <= 1:
        return [np.arange(Q.size)]
    dQ = np.abs(np.diff(Q))
    thresh = np.maximum(abs_tol, rel_tol * np.maximum(1.0, np.abs(Q[:-1])))
    edges = np.flatnonzero(dQ > thresh) + 1
    return np.split(np.arange(Q.size), edges)

segments = find_constant_segments(Q_star_phys, rel_tol=2e-3, abs_tol=0.05)
print(f"[INFO] Found {len(segments)} constant segments in HeatLoad target.")


def make_init_population(seed_row: np.ndarray, ndim: int, seed: int) -> np.ndarray:
    rng = np.random.default_rng(seed)
    n_pop = popsize * ndim
    init_pop = np.empty((n_pop, ndim), dtype=float)
    init_pop[0] = np.clip(seed_row, lb_single, ub_single)
    for idx in range(1, n_pop):
        r = rng.random(ndim)
        init_pop[idx] = lb_single + r * (ub_single - lb_single)
    return init_pop

def optimize_single_step(t_idx: int, seed_row: np.ndarray, seed: int) -> np.ndarray:
    yQ_t = float(yQ_target_scaled[t_idx])
    ndim = N_DECISION_FEATURES
    init_pop = make_init_population(seed_row, ndim, seed)


    progress = {"gen": 0, "bestJ": np.inf}
    def _callback(best_x, convergence):
        progress["gen"] += 1
        J = objective_single_step(best_x, yQ_t)
        dJ = (progress["bestJ"] - J) if np.isfinite(progress["bestJ"]) else np.nan
        if J < progress["bestJ"]:
            progress["bestJ"] = J
        if (progress["gen"] % PRINT_EVERY == 0) or (progress["gen"] == 1):
            print(f"[DE:t={t_idx:4d}] gen {progress['gen']:3d} | J_best={J:.6e} | ΔJ={(0 if np.isnan(dJ) else dJ):.3e} | conv={convergence:.3e}")
        return False  

    result = differential_evolution(
        func=lambda x: objective_single_step(x, yQ_t),
        bounds=bounds_decision_single,
        strategy=strategy,
        maxiter=maxiter,
        popsize=popsize,
        mutation=mutation,
        recombination=recomb,
        seed=seed,
        updating="deferred",   
        workers=1,           
        polish=True,
        init=init_pop,
        callback=_callback,
    )

    print(f"[DE:t={t_idx:4d}] done | nit={result.nit:3d} | nfev={result.nfev:5d} | J*={result.fun:.6e} | status: {result.message}")

    # Return projected optimum
    return hard_project_coal_decision_scaled_row(result.x)

X_best_decision_scaled = np.empty((N_STEPS, N_DECISION_FEATURES), dtype=float)

for segment_idx, segment_indices in enumerate(segments):
    #independent seed & independent seed row per segment
    seg_seed = 12345 + segment_idx
    seed_segment_row = np.clip(np.mean(X_scaled_decision[segment_indices, :], axis=0), lb_single, ub_single)
    mid_timestep = int(segment_indices[len(segment_indices)//2])
    optimal_row = optimize_single_step(t_idx=mid_timestep, seed_row=seed_segment_row, seed=seg_seed)
    X_best_decision_scaled[segment_indices, :] = optimal_row

for t in range(N_STEPS):
    X_best_decision_scaled[t, :] = hard_project_coal_decision_scaled_row(X_best_decision_scaled[t, :])

n_unique = np.unique(X_best_decision_scaled, axis=0).shape[0]
print(f"[DEBUG] Unique decision rows: {n_unique} (segments: {len(segments)})")

x_best_scaled_flat = matrix_to_flat(X_best_decision_scaled)



In [ ]:
# G) Export optimised schedule + predictions
# =========================
if "X_best_decision_scaled" not in locals():
    raise RuntimeError("no optimised decision schedule found. did section f run?")

#insert fixed t_amb and build full 12-col scaled inputs
X_best_full_scaled = np.empty((N_STEPS, N_FEATURES_FULL), dtype=float)
for k, j in enumerate(decision_feature_indices):
    X_best_full_scaled[:, j] = X_best_decision_scaled[:, k]
X_best_full_scaled[:, i_Tamb] = Tamb_fixed_scaled

#invert inputs to physical units
X_best_phys = X_scaler.inverse_transform(X_best_full_scaled).astype(np.float64)

#model predictions at the best schedule (scaled → physical)
with torch.no_grad():
    y_scaled_best = model(
        torch.from_numpy(X_best_full_scaled).to(device=device, dtype=DEFAULT_DTYPE)
    ).cpu().numpy()

Y_best_phys    = Y_scaler.inverse_transform(y_scaled_best).astype(np.float64)
yQ_best_phys   = Y_best_phys[:, 0]
yEff_best_phys = Y_best_phys[:, 1]

#build output dataframe (numeric) + predictions
df_out = pd.DataFrame(X_best_phys, columns=input_names)
df_out["HeatLoad"]   = yQ_best_phys
df_out["Efficiency"] = yEff_best_phys

#add sumcoal = coal1+...+coal5
coal_cols = ["Coal1", "Coal2", "Coal3", "Coal4", "Coal5"]
df_out["SumCoal"] = df_out[coal_cols].sum(axis=1).astype(np.float64)

#save to csv with fixed-point formatting (no scientific notation)
OUT_CSV = "Sim_Test5_optimised.csv"
df_out.to_csv(OUT_CSV, index=False, float_format="%.6f")


pd.set_option("display.float_format", "{:.2f}".format)
print(df_out.iloc[7, 14])
#print(df_out.iloc[14, 14])
#print(df_out.iloc[15, 14])
print(df_out.iloc[28, 14])
#print(df_out.iloc[40, 14])
print(df_out.iloc[56, 14])
print(df_out.iloc[56, 13])
#print(df_out.iloc[71, 14])
print(df_out.iloc[78, 14])
#print(df_out.iloc[84, 14])
print(df_out.iloc[92, 14])